# Init

In [ ]:
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pathlib import Path
from random import randint
import json
%cd ..
%load_ext autoreload

In [ ]:
%autoreload 2
from src.MapData import MapData, min_max_dist, triangle_corr, get_triangular_kernel
from src.LineFinder import LineFinder

from src.plotting_functions import plot_embedding, plot_single_variable_map

# Data

In [ ]:
map_data = MapData(            
    './data/2022_03_22_P56B_307x532.libsdata',
    overwrite=False
)
map_data.get_metadata()
map_data.load_wavelenths()
map_data.load_all_data()

## Preprocessing

In [ ]:
CM = 2.54

In [ ]:
fig, ax = plt.subplots(1,figsize=(16/CM,16/CM))
ax.plot(
    map_data.wvl,
    map_data.spectra[randint(0,len(map_data.spectra)),:]
)

In [ ]:
map_data.trim_spectra(64)
map_data.estimate_systemic_noise()

map_data.get_baseline(
    min_window_size=50,
    smooth_window_size=100
)
map_data.baseline_correct()

map_data.upsample_spectra()
if map_data.systemic_noise_spectrum is not None:
    map_data.denoise_spectra(
        file_name_supplement='demo',
        threshold=np.std(map_data.systemic_noise_spectrum)
    )

maxima_spectrum = map_data.spectra.max(axis=0)